# Question 1 - Credit Default Swap (Fifth to default)

In [178]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import scipy.linalg as lag
import scipy.stats as sc_stat
import scipy as sc

## Initialize Variables 

In [179]:
N_asset = 10                                                            #Number of assets
r = 0.04                                                                #Risk free rate
T = 5                                                                   #time
rho = 0.8                                                               #Cross Correlation 
def_Int = [0.05, 0.01, 0.05, 0.05,0.01, 0.1, 0.01, 0.09, 0.1, 0.02]     #Default Intensities
rec_rate = [0.1, 0.1, 0.3, 0.1, 0.3, 0.1, 0.2, 0.2, 0.1, 0.1]           #Recovery rate
def_asset_threshold = 5                                                 # Number of assets to default for CDS payout
N =20000#00                                                              #Simulations

## Main Code Block 

### Generate Random Variables and default Times

In [180]:
#Generate correlated Uniform RVs : Generated only once since Solver to be used

Corr_mat = [[rho]*N_asset for i in range(N_asset)]                                          #Generate Correlation matrix
for i in range(N_asset):
    Corr_mat[i][i] = 1

A = lag.cholesky(Corr_mat,lower=True)                                                       #Cholsky Decomposition to find A
Z = [np.random.normal(0,1,N) for i in range(N_asset)]                                       #Generate independen    t normal 
X = np.matmul(A,Z)                                                                          #Get Correlated Multivariate Normal

#Get Correlated Uniform Random Variables
Corr_U = [[sc_stat.norm.cdf(X[i,j]) for j in range(N)] for i in range(N_asset)]

#Assuming an exponential model for default intensities and usign the CDF for that to inverse the uniform random variables.
def_times = np.array([[sc_stat.expon.ppf(Corr_U[i][j],scale = 1/def_Int[i]) for j in range(N)] for i in range(N_asset)])
N_def_before_T = [sum([1 if i < T else 0 for i in def_times[:,j]]) for j in range(N)]       #Number of defaults before Time T
Exer_time = np.array([sorted([i if i < T else 0 for i in def_times[:,j]]) for j in range(N)])         #exercise time 

### Function to caclulate swap value for given S

In [181]:
#create function to calculate the value of the Swap
def get_swap_value(s):
    s=s/10000
    V_value = [0]*N                                                                         #Initilize Arrays
    V_prot = [0]*N
    Value = [0]*N
    for j in range(N):                                                                      #Loop through each scenario
        
        if N_def_before_T[j]<5:                                                             #If no. of defaults are below 5 then no exercise    
            #No exercise 
            V_value[j] = 0
            V_prot[j] = sum((N_asset * s * np.exp(-r*i) for i in range(T)))
        else:
            #Excerise
            tau = Exer_time[j,T - N_def_before_T[j]-1]                                      #Calculate the 5th default time before time T 
            firm_index = list(def_times[:,j]).index(tau)                                    #Find the firm that defaulted
            V_value[j] = (1- rec_rate[firm_index])*np.exp(-r*tau)                           
            whole_tau = np.floor(tau)
            V_prot[j] =sum((N_asset * s * np.exp(-r*i) for i in range(int(whole_tau)))) + (N_asset * s * np.exp(-r*whole_tau)*(tau-whole_tau))
        
        Value[j] = V_value[j] - V_prot[j]
    return np.mean(Value)

### Find the root of the function

In [216]:
rt = sc.optimize.root(get_swap_value,0)
print(rt.x[0],"bps")

32.395897285286296 bps
